In [2]:
import shap
import numpy as np
import pandas as pd
import xgboost as xgb
import statsmodels.api as sm
from math import sqrt
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

/Users/winke/opt/anaconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# United States of America

In [3]:
#Load in data
#Import csv and remove non-numerical variables
df = pd.read_csv('weekly_new.csv')
df = df.drop(['name', 'year', 'week', 'latitude', 'longitude', 'espg'] , axis=1)
df = df[['counts_week', 'country', 'dem_mean', 'cycle_length', 'lst_mean', 'pop_sum', 'dem_std', 'build_area', 'street_length_total', 
          'ndvi_mean', 'restaurants', 'dist_to_bikePOI', '3_way_int_count', 'bike_points', 'daily_shops', 'median_speed', 'business_shops',
           'traffic_signals', 'dist_to_edu', 'dist_to_train', 'streets_per_node_avg', 'circuity_avg', 'lc_entropy', 'bus_stops', 'dist_to_greenspace']]

country = df[df['country'] == 'USA']

#Create dependent and independent variable
y = country.loc[:,'counts_week']
X = country.drop(['counts_week', 'country'], axis=1)

# Normalize dependent variable
scaler = StandardScaler()
data_scaled = scaler.fit_transform(y.values.reshape(-1, 1))
y = pd.Series(data_scaled.ravel())

In [4]:
#Create traintestsplit for machine learning models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

## Linear Regression (OLS)

In [7]:
#Add a constant to the data and run OLS regression
X_const = sm.add_constant(X)
X_const.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
model = sm.OLS(y, X_const).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     7.585
Date:                Mon, 13 Feb 2023   Prob (F-statistic):           3.07e-18
Time:                        15:11:16   Log-Likelihood:                -310.76
No. Observations:                 270   AIC:                             669.5
Df Residuals:                     246   BIC:                             755.9
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.4368 

In [8]:
#Check multicolinearity with VIF
vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

print(vif)

           VIF              features
0   167.879689                 const
1     1.376480              dem_mean
2     1.378415          cycle_length
3     1.476362              lst_mean
4     1.846891               pop_sum
5     1.193945               dem_std
6     3.014178            build_area
7     3.791934   street_length_total
8     1.762835             ndvi_mean
9     5.351883           restaurants
10    1.387129       dist_to_bikePOI
11    2.261218       3_way_int_count
12    1.684011           bike_points
13    2.339906           daily_shops
14    1.228074          median_speed
15    3.555449        business_shops
16    3.699794       traffic_signals
17    1.362589           dist_to_edu
18    1.281733         dist_to_train
19    3.688623  streets_per_node_avg
20    2.650224          circuity_avg
21    1.343562            lc_entropy
22    1.800952             bus_stops
23    1.257672    dist_to_greenspace


## Random Forest

In [4]:
# Define the parameter grid to search
param_grid = {'n_estimators': [50, 100, 200, 300, 400, 500],
            'max_depth': [5, 10, 15, 20, 25],
            'min_samples_split': [2, 5, 10]}

# Create the random forest model
rf = RandomForestRegressor()

# Create the K-fold cross-validation object
kf = KFold(n_splits=5)

# Create the grid search object
grid_search = GridSearchCV(rf, param_grid, cv=kf, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

#Create model from best parameters
best_params = grid_search.best_params_
model_rf = RandomForestRegressor(**best_params)

#Run the model on the test split of the data
model_rf.fit(X_train, y_train)
y_pred = grid_search.predict(X_test)

#Calculate model statistics and print them
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("R-squared: ", r2)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAE: ", mae)

df_results = pd.DataFrame()
df_results['R-squared'] = r2
df_results['MSE'] = mse
df_results['RMSE'] = rmse
df_results['MAE'] = mae
df_results

Best parameters:  {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}
Best score:  -0.7147134253500317
R-squared:  0.260865909322722
MSE:  0.13677493469853944
RMSE:  0.3698309542190045
MAE:  0.2948775276264462


In [ ]:
#Using SHAP to explain things
explainer = shap.Explainer(model_rf, X_train)
shap_values = explainer(X)

# shap_values.display_data = shap.datasets.adult(display=True)[0].values
shap.plots.bar(shap_values)

shap.plots.beeswarm(shap_values)

shap.plots.beeswarm(shap_values.abs, color="shap_red")

shap.plots.bar(shap_values[1])

shap.plots.scatter(shap_values[:,"ndvi_mean"], color=shap_values)

## XGBoost

In [6]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold

# Define the parameter grid to search
param_grid = {'n_estimators': [50, 100, 200, 250, 300, 400],
              'max_depth': [3, 5, 10, 15],
              'learning_rate': [0.1, 0.3, 0.5, 1],
              'subsample': [0.8, 1],
              'colsample_bytree': [0.8, 1]
              }

# Create the XGBoost model
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Create the K-fold cross-validation object
kf = KFold(n_splits=5)

# Create the grid search object
grid_search = GridSearchCV(xgb_reg,
                param_grid=param_grid,
                cv=kf,
                scoring='neg_mean_squared_error',
                verbose=True)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

#Create model from best parameters
best_params = grid_search.best_params_
model_xgb = xgb.XGBRegressor(**best_params)

# Get the predictions of the model on the test data
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)

# Calculate the R-squared, RMSE, MSE and F1-score
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("R-squared: ", r2)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAE: ", mae)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits
Best parameters:  {'colsample_bytree': 0.8, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1}
Best score:  -0.7519803897409669
R-squared:  0.5177885117327485
MSE:  0.08923204280593246
RMSE:  0.2987173292695495
MAE:  0.24309479402165282


In [ ]:
#Using SHAP to explain things
explainer = shap.Explainer(model_xgb, X_train)
shap_values = explainer(X)

# shap_values.display_data = shap.datasets.adult(display=True)[0].values
shap.plots.bar(shap_values)

shap.plots.beeswarm(shap_values)

shap.plots.beeswarm(shap_values.abs, color="shap_red")

shap.plots.bar(shap_values[1])

shap.plots.scatter(shap_values[:,"ndvi_mean"], color=shap_values)

## SVM

In [ ]:
# Define the parameter grid to search
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'],
              'kernel': ['linear', 'rbf']
              }

# Create the SVM model
svm_reg = SVR()

# Create the K-fold cross-validation object
kf = KFold(n_splits=5)

# Create the grid search object
grid_search = GridSearchCV(svm_reg,
                param_grid=param_grid,
                cv=kf,
                scoring='neg_mean_squared_error',
                verbose=True)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

#Create model from best parameters
best_params = grid_search.best_params_
model_svm = SVR(**best_params)

# Get the predictions of the model on the test data
model_svm.fit(X_train, y_train)
y_pred = model_svm.predict(X_test)

# Calculate the R-squared, RMSE, MSE and F1-score
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("R-squared: ", r2)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAE: ", mae)

In [ ]:
#Using SHAP to explain things
explainer = shap.Explainer(model_svm, X_train)
shap_values = explainer(X)

# shap_values.display_data = shap.datasets.adult(display=True)[0].values
shap.plots.bar(shap_values)

shap.plots.beeswarm(shap_values)

shap.plots.beeswarm(shap_values.abs, color="shap_red")

shap.plots.bar(shap_values[1])

shap.plots.scatter(shap_values[:,"ndvi_mean"], color=shap_values)